In [1]:
import json
import os
from PIL import Image

In [2]:
def load_annotations(json_file):
    """Charge le fichier JSON d'annotations VIA et extrait les annotations."""
    with open(json_file, 'r') as f:
        data = json.load(f)
    # Si le format est VIA, les annotations se trouvent dans "_via_img_metadata"
    if "_via_img_metadata" in data:
        return data["_via_img_metadata"]
    return data

In [3]:
def get_class_mapping(annotations):
    """
    Génère un mapping (classe combinée -> id) à partir des annotations.
    Pour chaque région, on parcourt les attributs pour extraire la main classe et la sous-classe.
    Si la clé est "4x2", elle est convertie en "2x4" selon la demande.
    La classe finale est sous la forme "mainclasse_sousclasse" (ex: "2x2_Rouge").
    """
    classes = set()
    for key, value in annotations.items():
        regions = value.get("regions", [])
        # Certains fichiers VIA stockent les régions dans un dictionnaire
        if isinstance(regions, dict):
            regions = regions.values()
        for region in regions:
            region_attr = region.get("region_attributes", {})
            # On attend qu'une seule clé ait une valeur non vide
            for main_key, sub_class in region_attr.items():
                if sub_class:  # si la sous-classe n'est pas vide
                    # Conversion de "4x2" en "2x4" pour respecter la nomenclature souhaitée
                    main_class = "2x4" if main_key == "4x2" else main_key
                    combined_class = f"{main_class}_{sub_class}"
                    classes.add(combined_class)
                    break  # on considère la première valeur non vide
    # Crée un mapping trié pour un ordre stable
    class_to_id = {cls: idx for idx, cls in enumerate(sorted(classes))}
    return class_to_id

In [4]:
def convert_annotations(annotations, images_dir, output_dir, class_mapping):
    """
    Convertit les annotations VIA en fichiers texte au format YOLO.
    Pour chaque région, la classe est déterminée en combinant le type de brique et sa couleur.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for key, value in annotations.items():
        filename = value.get("filename")
        image_path = os.path.join(images_dir, filename)
        if not os.path.exists(image_path):
            print(f"Image {image_path} non trouvée, passage...")
            continue
        
        # Ouvre l'image pour récupérer sa taille
        with Image.open(image_path) as img:
            img_width, img_height = img.size
        
        # Fichier de sortie (même nom que l'image, extension .txt)
        base_filename = os.path.splitext(filename)[0]
        output_file = os.path.join(output_dir, base_filename + ".txt")
        
        lines = []
        regions = value.get("regions", [])
        if isinstance(regions, dict):
            regions = regions.values()
        for region in regions:
            shape_attr = region.get("shape_attributes", {})
            region_attr = region.get("region_attributes", {})
            
            combined_class = None
            for main_key, sub_class in region_attr.items():
                if sub_class:
                    main_class = "2x4" if main_key == "4x2" else main_key
                    combined_class = f"{main_class}_{sub_class}"
                    break
            if combined_class is None:
                continue  # pas d'annotation pour cette région
            
            class_id = class_mapping.get(combined_class)
            if class_id is None:
                print(f"Classe {combined_class} non trouvée dans le mapping.")
                continue
            
            # On traite ici uniquement les rectangles
            if shape_attr.get("name") == "rect":
                x = shape_attr.get("x", 0)
                y = shape_attr.get("y", 0)
                width = shape_attr.get("width", 0)
                height = shape_attr.get("height", 0)
                
                # Conversion au format YOLO : coordonnées normalisées
                x_center = (x + width/2) / img_width
                y_center = (y + height/2) / img_height
                width_norm = width / img_width
                height_norm = height / img_height
                
                line = f"{class_id} {x_center:.6f} {y_center:.6f} {width_norm:.6f} {height_norm:.6f}"
                lines.append(line)
            # Vous pouvez ajouter ici la gestion d'autres formes si nécessaire.
            
        with open(output_file, "w") as f:
            f.write("\n".join(lines))
        print(f"Annotations converties pour {filename} enregistrées dans {output_file}")

In [5]:
# Chemin vers votre fichier JSON d'annotations
json_file = "../data/json_Via/Annotation_train_set.json"
# Dossier contenant les images référencées dans le JSON
images_dir = "../images/images_brut/train_set"
# Dossier où seront enregistrées les annotations au format YOLO
output_dir = "../data/yolo_annotation"

# Charge les annotations
annotations = load_annotations(json_file)
# print(type(annotations))
# for key, value in annotations.items():
#     print(key, type(value), value)

# Génère le mapping classe -> id
class_mapping = get_class_mapping(annotations)

# Sauvegarde le mapping dans un fichier classes.txt (utile pour la configuration YOLO)
classes_file = os.path.join(output_dir, "classes.txt")
with open(classes_file, "w") as f:
    for cls, idx in class_mapping.items():
        f.write(f"{cls}\n")
print("Mapping des classes sauvegardé dans", classes_file)

# Conversion des annotations
convert_annotations(annotations, images_dir, output_dir, class_mapping)


Mapping des classes sauvegardé dans ../data/yolo_annotation/classes.txt
Annotations converties pour IMG-20250325-WA0003.jpg enregistrées dans ../data/yolo_annotation/IMG-20250325-WA0003.txt
Annotations converties pour IMG-20250325-WA0004.jpg enregistrées dans ../data/yolo_annotation/IMG-20250325-WA0004.txt
Annotations converties pour IMG-20250325-WA0005.jpg enregistrées dans ../data/yolo_annotation/IMG-20250325-WA0005.txt
Annotations converties pour IMG-20250325-WA0006.jpg enregistrées dans ../data/yolo_annotation/IMG-20250325-WA0006.txt
Annotations converties pour IMG-20250325-WA0007.jpg enregistrées dans ../data/yolo_annotation/IMG-20250325-WA0007.txt
Annotations converties pour IMG-20250325-WA0008.jpg enregistrées dans ../data/yolo_annotation/IMG-20250325-WA0008.txt
Annotations converties pour IMG-20250325-WA0009.jpg enregistrées dans ../data/yolo_annotation/IMG-20250325-WA0009.txt
Annotations converties pour IMG-20250325-WA0010.jpg enregistrées dans ../data/yolo_annotation/IMG-2025